<a href="https://colab.research.google.com/github/Abidt2002/AI-EduMaster-Platform/blob/main/AI_EduMaster_Platform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
!pip install streamlit pyngrok transformers langchain faiss-cpu sentence-transformers pypdf bcrypt sqlite-utils accelerate bitsandbytes


In [37]:
import getpass, os, subprocess

# 🔒 Step 1: Enter your real ngrok token securely (it will stay hidden)
NGROK_AUTH_TOKEN = getpass.getpass("Enter your real ngrok authtoken (hidden): ")

# 🔒 Step 2: Store it temporarily and configure ngrok silently
os.environ["NGROK_AUTH_TOKEN"] = NGROK_AUTH_TOKEN
subprocess.run(["ngrok", "config", "add-authtoken", NGROK_AUTH_TOKEN], stdout=subprocess.DEVNULL)
print("✅ ngrok token configured securely!")


Enter your real ngrok authtoken (hidden): ··········
✅ ngrok token configured securely!


In [38]:
from pyngrok import ngrok
ngrok.kill()
public_url = ngrok.connect(8501)
print("🌐 Your Live App Link:", public_url)


🌐 Your Live App Link: NgrokTunnel: "https://fcb4fa93a75c.ngrok-free.app" -> "http://localhost:8501"


In [39]:
%%writefile app.py
import streamlit as st
import sqlite3, bcrypt, os, time, pickle, numpy as np
from datetime import datetime
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline

# ----- Config -----
DATA_DIR = "data"
DB_DIR = "db"
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(DB_DIR, exist_ok=True)

# ----- Models (load once) -----
@st.cache_resource
def load_models():
    emb_model = SentenceTransformer("all-MiniLM-L6-v2")
    qa_pipe = pipeline("text2text-generation", model="google/flan-t5-base", device_map="auto" if False else None)
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    return emb_model, qa_pipe, summarizer

emb_model, qa_pipe, summarizer = load_models()

# ----- DB Setup -----
conn = sqlite3.connect(os.path.join(DB_DIR, "users_activities.db"), check_same_thread=False)
cur = conn.cursor()
cur.execute("""CREATE TABLE IF NOT EXISTS users (
    username TEXT PRIMARY KEY,
    password BLOB NOT NULL,
    role TEXT NOT NULL
)""")
cur.execute("""CREATE TABLE IF NOT EXISTS activities (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    timestamp TEXT,
    username TEXT,
    role TEXT,
    action TEXT,
    details TEXT
)""")
conn.commit()

# ----- Utilities -----
def log_activity(username, role, action, details=""):
    cur.execute("INSERT INTO activities (timestamp, username, role, action, details) VALUES (?, ?, ?, ?, ?)",
                (datetime.utcnow().isoformat(), username, role, action, details))
    conn.commit()

def hash_password(password: str):
    return bcrypt.hashpw(password.encode(), bcrypt.gensalt())

def verify_password(password: str, hashed: bytes):
    return bcrypt.checkpw(password.encode(), hashed)

def add_user(username, password, role):
    cur.execute("INSERT INTO users (username, password, role) VALUES (?, ?, ?)", (username, hash_password(password), role))
    conn.commit()

def authenticate(username, password):
    cur.execute("SELECT password, role FROM users WHERE username=?", (username,))
    row = cur.fetchone()
    if row and verify_password(password, row[0]):
        return row[1]
    return None

# ----- Vector store helpers (per teacher) -----
# We store for each teacher: index file (faiss) + metadata list (texts)
def _chunks_from_text(text, chunk_size=800, overlap=100):
    chunks = []
    i = 0
    while i < len(text):
        chunk = text[i:i+chunk_size]
        chunks.append(chunk)
        i += chunk_size - overlap
    return chunks

def extract_text_from_pdf(path):
    text = ""
    reader = PdfReader(path)
    for p in reader.pages:
        try:
            t = p.extract_text()
            if t:
                text += "\n" + t
        except:
            continue
    return text

def build_teacher_index(username, filepath):
    # extract
    fulltext = extract_text_from_pdf(filepath)
    chunks = _chunks_from_text(fulltext)
    # embeddings
    embeddings = emb_model.encode(chunks, show_progress_bar=False, convert_to_numpy=True)
    # normalize for cosine
    faiss.normalize_L2(embeddings)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatIP(dim)
    index.add(embeddings)
    # save index and metadata
    idx_path = os.path.join(DATA_DIR, f"{username}_index.faiss")
    faiss.write_index(index, idx_path)
    meta_path = os.path.join(DATA_DIR, f"{username}_meta.pkl")
    with open(meta_path, "wb") as f:
        pickle.dump(chunks, f)
    return True

def load_teacher_index(username):
    idx_path = os.path.join(DATA_DIR, f"{username}_index.faiss")
    meta_path = os.path.join(DATA_DIR, f"{username}_meta.pkl")
    if not os.path.exists(idx_path) or not os.path.exists(meta_path):
        return None, None
    index = faiss.read_index(idx_path)
    with open(meta_path, "rb") as f:
        chunks = pickle.load(f)
    return index, chunks

def teacher_similarity_search(username, query, top_k=3):
    index, chunks = load_teacher_index(username)
    if index is None:
        return []
    q_emb = emb_model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(q_emb)
    D, I = index.search(q_emb, top_k)
    results = []
    for idx in I[0]:
        if idx < len(chunks):
            results.append(chunks[idx])
    return results

# ----- Streams / UI -----
st.set_page_config(page_title="AI Education Platform", layout="wide")
st.title("🎓 AI-Powered Educational Platform (Colab)")

menu = ["Login", "Register"]
choice = st.sidebar.selectbox("Menu", menu)

if "username" not in st.session_state:
    st.session_state.username = None
if "role" not in st.session_state:
    st.session_state.role = None

# ----- Register -----
if choice == "Register":
    st.subheader("Create a new account")
    new_user = st.text_input("Username")
    new_pass = st.text_input("Password", type="password")
    role = st.selectbox("Role", ["Student", "Teacher", "Admin"])
    if st.button("Register"):
        try:
            add_user(new_user, new_pass, role)
            st.success("✅ Account created. Please login.")
            log_activity(new_user, role, "register", "")
        except Exception as e:
            st.error("Username already exists or invalid. " + str(e))

# ----- Login -----
elif choice == "Login":
    st.subheader("Login")
    username = st.text_input("Username")
    password = st.text_input("Password", type="password")
    if st.button("Login"):
        role = authenticate(username, password)
        if role:
            st.session_state.username = username
            st.session_state.role = role
            st.success(f"Welcome {username} ({role})")
            log_activity(username, role, "login", "")
        else:
            st.error("❌ Invalid username or password")

# ----- Dashboards -----
if st.session_state.role == "Teacher":
    username = st.session_state.username
    st.header(f"👩‍🏫 Teacher Dashboard — {username}")

    st.subheader("Upload Study Material (PDF)")
    uploaded = st.file_uploader("Upload one or more PDFs", type=["pdf"], accept_multiple_files=True)
    if uploaded:
        for up in uploaded:
            save_path = os.path.join(DATA_DIR, f"{username}__{up.name}")
            with open(save_path, "wb") as f:
                f.write(up.getbuffer())
            # build index for this file (append to teacher index: for simplicity we rebuild using the latest file only)
            # For production you may merge all teacher files; here we rebuild from latest file
            build_teacher_index(username, save_path)
            st.success(f"Saved and indexed: {up.name}")
            log_activity(username, "Teacher", "upload", up.name)

    st.subheader("AI Tools")
    # Summarize prompt
    summary_q = st.text_input("Ask the system to summarize (e.g. 'Summarize chapter 1'):")
    if st.button("Summarize from uploaded material") and summary_q:
        ctxs = teacher_similarity_search(username, summary_q, top_k=3)
        if not ctxs:
            st.warning("No teacher material indexed yet. Please upload PDFs above.")
        else:
            ctx = "\n".join(ctxs)
            prompt = f"Summarize the following content in 3-6 sentences:\n\n{ctx}"
            res = summarizer(prompt, max_length=200, truncation=True)
            out = res[0]["summary_text"]
            st.text_area("Summary", out, height=200)
            log_activity(username, "Teacher", "summarize", summary_q)

    # Generate Quiz
    quiz_topic = st.text_input("Enter a topic (or keyword) to generate 5 MCQs:")
    if st.button("Generate Quiz") and quiz_topic:
        ctxs = teacher_similarity_search(username, quiz_topic, top_k=3)
        if not ctxs:
            st.warning("No teacher material indexed yet. Please upload PDFs.")
        else:
            ctx = "\n".join(ctxs)
            q_prompt = (f"Based on the content below, generate 5 multiple-choice questions (each with 4 options "
                        f"and indicate the correct option) about: {quiz_topic}\n\nContent:\n{ctx}\n\nOutput format:\n1) Q? \nA) ... B) ... C) ... D) ... Answer: B\n...")
            res = qa_pipe(q_prompt, max_length=512)
            quiz_text = res[0]["generated_text"]
            st.text_area("AI Generated Quiz (raw)", quiz_text, height=300)
            # Save quiz to disk
            qfile = os.path.join("data", f"{username}__quiz__{int(time.time())}.txt")
            with open(qfile, "w") as f:
                f.write(quiz_text)
            log_activity(username, "Teacher", "generate_quiz", quiz_topic)

    # View teacher's indexed status
    idx, meta = load_teacher_index(username)
    if idx is None:
        st.info("No index found yet for your uploads.")
    else:
        st.success("✅ Your material is indexed and ready for Student queries.")

    # Optionally list uploaded files
    st.subheader("Uploaded files")
    files = [f for f in os.listdir(DATA_DIR) if f.startswith(username + "__")]
    st.write(files)

elif st.session_state.role == "Student":
    username = st.session_state.username
    st.header(f"🎓 Student Dashboard — {username}")
    st.subheader("Ask a question from your teacher's material")
    teacher_name = st.text_input("Enter teacher username to query (exact):")
    question = st.text_input("Your question (e.g. 'Summarize chapter 1'):")
    if st.button("Ask") and teacher_name and question:
        idx, meta = load_teacher_index(teacher_name)
        if idx is None:
            st.error("That teacher has not uploaded or indexed any material yet.")
        else:
            # combine top contexts
            ctxs = teacher_similarity_search(teacher_name, question, top_k=3)
            ctx = "\n".join(ctxs)
            prompt = f"Context: {ctx}\n\nQuestion: {question}\nAnswer concisely:"
            res = qa_pipe(prompt, max_length=256)
            ans = res[0]["generated_text"]
            st.text_area("AI Answer", ans, height=200)
            log_activity(username, "Student", "ask_question", f"teacher={teacher_name}; q={question}")

    st.subheader("Practice Quizzes (auto-generated)")
    # list quizzes for all teachers (simple)
    qlist = [f for f in os.listdir(DATA_DIR) if "__quiz__" in f]
    if qlist:
        chosen = st.selectbox("Select a quiz file", qlist)
        if st.button("Load Quiz"):
            with open(os.path.join(DATA_DIR, chosen), "r") as f:
                st.text_area("Quiz content", f.read(), height=300)
            log_activity(username, "Student", "load_quiz", chosen)
    else:
        st.info("No quizzes available yet. Ask your teacher to generate one.")

elif st.session_state.role == "Admin":
    username = st.session_state.username
    st.header("🧑‍💼 Admin Dashboard")
    st.subheader("Users")
    cur.execute("SELECT username, role FROM users")
    users = cur.fetchall()
    st.table(users)

    st.subheader("Activity Logs (most recent 200)")
    cur.execute("SELECT timestamp, username, role, action, details FROM activities ORDER BY id DESC LIMIT 200")
    logs = cur.fetchall()
    st.dataframe(logs)

    # Optionally download logs
    if st.button("Export logs to file"):
        outp = os.path.join(DB_DIR, "activities_export.csv")
        import csv
        with open(outp, "w", newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(["timestamp", "username", "role", "action", "details"])
            writer.writerows(logs)
        st.success(f"Exported to {outp}")

else:
    st.info("Please Login or Register from the sidebar. Roles available: Student, Teacher, Admin")
    st.write("Recommended demo flow:")
    st.write("1) Register a Teacher account and upload a PDF in Teacher dashboard.")
    st.write("2) Register a Student account and query that teacher's material.")

st.sidebar.markdown("---")
st.sidebar.caption("Secure role-based AI Education • HF models • FAISS local retrieval")


Overwriting app.py


In [40]:
# start streamlit (background)
get_ipython().system_raw("streamlit run app.py &")

# connect ngrok and print URL
from pyngrok import ngrok
ngrok.kill()
public_url = ngrok.connect(8501)
print("🌐 Your public URL ->", public_url)


🌐 Your public URL -> NgrokTunnel: "https://806a2076fce7.ngrok-free.app" -> "http://localhost:8501"
